# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-large-uncased")

/home/overman/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"].shuffle(seed=43)
eval_dataset = tokenized_datasets["test"].shuffle(seed=43)

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("models/bert-finetune-yelp", num_labels=5)

### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [5]:
model_dir = "models/bert-finetune-yelp-2"

from transformers import TrainingArguments
num_epochs = 2
batch_size = 13
warmup_steps = int(0.1 * (len(train_dataset) * num_epochs // batch_size))

training_args = TrainingArguments(
    output_dir=model_dir,
    evaluation_strategy="steps",
    eval_steps=1000,  # 更频繁地评估
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_steps=1000,  # 更频繁地记录
    weight_decay=0.1,  # 增加权重衰减
    warmup_steps=warmup_steps,
    fp16=True,
    save_steps=1000,  # 更频繁地保存
    save_total_limit=2,  # 限制保存的检查点数量
    learning_rate=2e-5,  # 降低学习率
    load_best_model_at_end=True,  # 加载验证集上表现最好的模型
    metric_for_best_model="eval_loss",  # 根据验证损失选择最佳模型
    greater_is_better=False,  # 损失越低越好
    gradient_accumulation_steps=2,  # 梯度累积
    lr_scheduler_type="cosine",  # 使用余弦学习率调度
)

In [6]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=1000,
evaluation_strategy=steps,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_l

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## 开始训练

### 实例化训练器（Trainer）

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
1000,0.610100,0.708587,0.694420
2000,0.611600,0.713346,0.694440
3000,0.610900,0.708096,0.692900
4000,0.623200,0.722619,0.691680
5000,0.627200,0.721743,0.688560
6000,0.617500,0.727931,0.691940
7000,0.625400,0.728096,0.690280
8000,0.633200,0.734139,0.685460
9000,0.634500,0.726283,0.685620
10000,0.637700,0.717859,0.689280


TrainOutput(global_step=50000, training_loss=0.6047294885253907, metrics={'train_runtime': 45718.7827, 'train_samples_per_second': 28.435, 'train_steps_per_second': 1.094, 'total_flos': 1.2115230526464e+18, 'train_loss': 0.6047294885253907, 'epoch': 2.0})

In [11]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64)

In [12]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.7031425833702087,
 'eval_accuracy': 0.69016,
 'eval_runtime': 326.5966,
 'eval_samples_per_second': 153.094,
 'eval_steps_per_second': 19.137,
 'epoch': 2.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [13]:
trainer.save_model(model_dir)

In [14]:
trainer.save_state()